In [1]:
import requests
from pprint import pprint
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
sns.set()
%matplotlib inline

In [6]:
db = pymysql.connect(host='localhost', user='root', database='admin_main', port = 3307, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

# Create a cursor object
cursorObject = db.cursor()

messages_Query = "select * from messages"
messages_meta_Query =  "select * from messages_meta"
order_Query = "select * from orders"

cursorObject.execute(messages_Query)
cursorObject.execute(messages_meta_Query)
cursorObject.execute(order_Query)

messages_df = pd.read_sql(messages_Query, db)
messages_meta_df = pd.read_sql(messages_meta_Query, db)
order_df = pd.read_sql(order_Query, db)

In [7]:
messages_new = messages_df.drop(messages_df.columns.difference(['id', 'conversation_id', 'session_id','recipient_id', 'sender_id', 'type', 'created_at', 'updated_at']), 1)
messagesmeta_new = messages_meta_df.drop(messages_meta_df.columns.difference(['message_id', 'value']), 1)
order_new = order_df.drop(order_df.columns.difference(['subscriber_id','status', 'created_at' 'updated_at']), 1)


In [8]:
import ast
messagesmeta_new.rename(columns={'message_id': 'id'}, inplace=True)

messages = pd.merge(messagesmeta_new,messages_new, on='id', how='inner')

messages_new = messages[messages.recipient_id == 0]
messages_new = messages_new[messages_new.type == 'postback']

messages_new['value']=messages_new['value'].apply(ast.literal_eval)

# c = 1
# for k in messages_new.value:
#     messages_new['new_value'] = k[c]['title']
#     c+=1

messages_new['payload'] = [d.get('payload') for d in messages_new.value]
messages_new['title'] = [d.get('title') for d in messages_new.value]

# Slice by Date and Activities

start_date = "2019-11-01 00:00:00"
end_date = "2019-11-30 23:59:59"
messages_new = messages_new[messages_new.created_at.between(start_date, end_date)]

# list = ['Get Started', 'Continue', 'Add More Products', 'Add to My Bag']

# messages_new = messages_new[messages_new['new_value'].isin(list)]

In [ ]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/mcdo_processmining.xlsx", engine='xlsxwriter')

messages_new.to_excel(writer, sheet_name='mcdo')

writer.save()